In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Hit_2.csv")
df.drop(columns = df.columns[0], inplace = True)

In [5]:
df.head()

,SongID,Song_x,Performer_x,Lyrics_x,Hit_x,Cleaned_Lyrics
0,'65 Love AffairPaul Davis,'65 Love Affair,Paul Davis,I was a car hop\rYou were into be-bop\rYou san...,False,car hop bebop sang wop diddy wop diddy wop doo...
1,'Til My Baby Comes HomeLuther Vandross,'Til My Baby Comes Home,Luther Vandross,Theres a whole lot of girls\r\nmessin around\r...,False,whole lot girls messin around trying get thing...
2,'Til Summer Comes AroundKeith Urban,'Til Summer Comes Around,Keith Urban,Another long summer's come and gone\r\nI don't...,False,another long summer 's come gone know always e...
3,'Til You Do Me RightAfter 7,'Til You Do Me Right,After 7,I was in love with you\r\nAnd gave my heart to...,False,love gave heart best keep satisfied took love ...
4,'TilThe Angels,'Til,The Angels,"Due to copyright restrictions, we are not auth...",False,due copyright restrictions authorized display ...


In [7]:
df.columns

Index(['SongID', 'Song_x', 'Performer_x', 'Lyrics_x', 'Hit_x',
       'Cleaned_Lyrics'],
      dtype='object')

In [9]:
df.rename(columns = {"Song_x" : "Song", "Performer_x" : "Performer", "Lyrics_x" : "Lyrics", "Hit_x" : "Hit"}, inplace = True)

In [11]:
df.head()

,SongID,Song,Performer,Lyrics,Hit,Cleaned_Lyrics
0,'65 Love AffairPaul Davis,'65 Love Affair,Paul Davis,I was a car hop\rYou were into be-bop\rYou san...,False,car hop bebop sang wop diddy wop diddy wop doo...
1,'Til My Baby Comes HomeLuther Vandross,'Til My Baby Comes Home,Luther Vandross,Theres a whole lot of girls\r\nmessin around\r...,False,whole lot girls messin around trying get thing...
2,'Til Summer Comes AroundKeith Urban,'Til Summer Comes Around,Keith Urban,Another long summer's come and gone\r\nI don't...,False,another long summer 's come gone know always e...
3,'Til You Do Me RightAfter 7,'Til You Do Me Right,After 7,I was in love with you\r\nAnd gave my heart to...,False,love gave heart best keep satisfied took love ...
4,'TilThe Angels,'Til,The Angels,"Due to copyright restrictions, we are not auth...",False,due copyright restrictions authorized display ...


In [13]:
df["Hit"] = df["Hit"].astype('int') 

# Handling Class Imblaance 

In [16]:
from imblearn.under_sampling import RandomUnderSampler

In [17]:
X = df['Cleaned_Lyrics']  # Features (lyrics)
y = df['Hit']

In [20]:
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

In [22]:
X_resampled, y_resampled = undersampler.fit_resample(X.values.reshape(-1, 1), y)

In [24]:
df_resampled = pd.DataFrame({'cleaned_lyrics': X_resampled.flatten(), 'hit': y_resampled})
print(f"Resampled dataset shape: {df_resampled['hit'].value_counts()}")

Resampled dataset shape: hit
0    4915
1    4915
Name: count, dtype: int64


In [26]:
df_resampled.head()

,cleaned_lyrics,hit
0,starry eyed came running arms cried let go kne...,0
1,jesus loves little children little children wo...,0
2,free know price pay could lover life leads thr...,0
3,neighbor carl lives next door pink flamingos p...,0
4,every day see face crack smile talk try take p...,0


# Word2Vec

In [27]:
from tensorflow.keras.layers import Embedding

2025-03-16 04:19:05.475786: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [45]:
from gensim.models import Word2Vec

In [31]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2025-03-16 15:26:28.331146: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [32]:
df_resampled.head()

,cleaned_lyrics,hit
0,starry eyed came running arms cried let go kne...,0
1,jesus loves little children little children wo...,0
2,free know price pay could lover life leads thr...,0
3,neighbor carl lives next door pink flamingos p...,0
4,every day see face crack smile talk try take p...,0


In [35]:
df_resampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9830 entries, 0 to 9829
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cleaned_lyrics  9818 non-null   object
 1   hit             9830 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 153.7+ KB


In [37]:
df_resampled["cleaned_lyrics"] = df_resampled["cleaned_lyrics"].astype('str')

In [39]:
df_resampled.head()

,cleaned_lyrics,hit
0,starry eyed came running arms cried let go kne...,0
1,jesus loves little children little children wo...,0
2,free know price pay could lover life leads thr...,0
3,neighbor carl lives next door pink flamingos p...,0
4,every day see face crack smile talk try take p...,0


In [41]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_resampled['cleaned_lyrics'])
X_tokenized = tokenizer.texts_to_sequences(df_resampled['cleaned_lyrics'])

In [47]:
sentences = [text.split() for text in df_resampled['cleaned_lyrics']]

# 2. Create Word2Vec Embedding Matrix
word2vec_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)
embedding_dim = 100
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

In [48]:
for word, index in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[index] = word2vec_model.wv[word]


In [49]:
max_len = 100
X_padded = pad_sequences(X_tokenized, padding='post', maxlen=max_len)

In [50]:
y = df_resampled["hit"]

In [59]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D
from gensim.models import Word2Vec

In [61]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=100,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(GlobalAveragePooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │     4,284,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,284,100 (16.34 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,284,100 (16.34 MB)

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [71]:
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step - accuracy: 0.6148 - loss: 0.6427 - val_accuracy: 0.6586 - val_loss: 0.6227
Epoch 2/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 15s 75ms/step - accuracy: 0.6433 - loss: 0.6302 - val_accuracy: 0.6510 - val_loss: 0.6310
Epoch 3/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 15s 74ms/step - accuracy: 0.6471 - loss: 0.6248 - val_accuracy: 0.6580 - val_loss: 0.6277
Epoch 4/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - accuracy: 0.6445 - loss: 0.6280 - val_accuracy: 0.6554 - val_loss: 0.6240
Epoch 5/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - accuracy: 0.6448 - loss: 0.6265 - val_accuracy: 0.6643 - val_loss: 0.6221
Epoch 6/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 15s 74ms/step - accuracy: 0.6535 - loss: 0.6220 - val_accuracy: 0.6484 - val_loss: 0.6310
Epoch 7/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 15s 74ms/step - accuracy: 0.6582 - loss: 0.6159 - val_accuracy: 0.6567 - val_loss: 0.6296
Epoch 8/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 15s 75ms/step - accuracy: 0.6578 - loss: 0.6157 - 

In [77]:
import re
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [79]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int) #convert probabilities to binary predictions

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_probs)

62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


In [81]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

Accuracy: 0.6409
Precision: 0.6313
Recall: 0.6560
F1-score: 0.6434
ROC AUC: 0.6906


In [83]:
import joblib 

In [85]:
model.save('lyrics_model.h5')
word2vec_model.save('lyrics_word2vec.model')
joblib.dump(tokenizer, 'lyrics_tokenizer.pkl')

['lyrics_tokenizer.pkl']

# Input for ML

In [56]:
import numpy as np

# Function to compute the average Word2Vec vector for a given song's lyrics
def average_word_vectors(sequences, embedding_matrix, embedding_dim):
    features = []
    for seq in sequences:
        word_vectors = [embedding_matrix[idx] for idx in seq if idx < len(embedding_matrix)]
        if len(word_vectors) > 0:
            avg_vector = np.mean(word_vectors, axis=0)
        else:
            avg_vector = np.zeros(embedding_dim)  # Handle empty lyrics
        features.append(avg_vector)
    return np.array(features)

In [58]:
X_features = average_word_vectors(X_padded, embedding_matrix, embedding_dim)

In [62]:
y = df_resampled["hit"].values

# ML - Random Forest

In [66]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

In [68]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

In [70]:
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.6052899287894201
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.60      0.61       995
           1       0.60      0.61      0.60       971

    accuracy                           0.61      1966
   macro avg       0.61      0.61      0.61      1966
weighted avg       0.61      0.61      0.61      1966



# ML - SVM

In [75]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

In [76]:
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))

SVM Accuracy: 0.6236012207527976
SVM Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.65      0.63       995
           1       0.62      0.60      0.61       971

    accuracy                           0.62      1966
   macro avg       0.62      0.62      0.62      1966
weighted avg       0.62      0.62      0.62      1966



# ML - XGBoost

In [80]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred_xgb))

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:33:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.6108850457782299
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.62      0.62       995
           1       0.61      0.60      0.60       971

    accuracy                           0.61      1966
   macro avg       0.61      0.61      0.61      1966
weighted avg       0.61      0.61      0.61      1966

